# Create SPECFEM3D Project Directories
1. Define function for making directories
1. Create directories for each event including the special directories for run0001 

### 1. Define function for making directories

In [ ]:
import os

def make_directory(fqdn):
    if os.path.isdir(fqdn):
        print ("The directory %s has already been created" % fqdn)
        return 0
    try:
        os.makedirs(fqdn, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % fqdn)
        print()
        return -1
    else:
        print ("Creation of the directory %s succeeded" % fqdn)
        print()
        return 0
    

### 2. Create directories for each event including the special directories for run0001

In [ ]:
import pickle

data_root_dir = '../../../data_notebooks'

num_events = 3
proj_name = 'Project_Beta_3E'
proj_root_path = '/Users/mcmac/Documents/Work/Bench/Groningen/data_notebooks/SPECFEM3D/project/'
proj_fqdn = proj_root_path + proj_name + '/'

cluster_event_bin_fqdn = '/quanta1/home/tcullison/DevGPU_specfem3d/bin'
cluster_event_utils_fqdn = '/quanta1/home/tcullison/DevGPU_specfem3d/utils'
cluster_proj_pyutils_fqdn = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
cluster_proj_scriptutils_fqdn = '/quanta1/home/tcullison/myscripts/specfem'

# list of directories need for every event
common_dir_list = ['/DATA',
                   '/OUTPUT_FILES/DATABASES_MPI',
                   '/SEM',
                   '/OBS',
                   '/SYN',
                   '/FILT_OBS',
                   '/FILT_SYN',]

# list of directories only needed for the primary run0001 dir
primary_dir_list = ['/INPUT_GRADIENT',
                    '/INPUT_KERNELS',
                    '/INPUT_MODEL',
                    '/OUTPUT_MODEL',
                    '/OUTPUT_SUM',
                    '/SMOOTH',
                    '/COMBINE',
                    '/topo']

# Create Project Dictionary
project_dict = {}
project_dict['NUM_EVENTS'] = num_events
project_dict['PROJ_NAME'] = proj_name
project_dict['PROJ_ROOT_PATH'] = proj_root_path
project_dict['PROJ_PYUTILS'] = cluster_proj_pyutils_fqdn
project_dict['PROJ_SCRIPTUTILS'] = cluster_proj_scriptutils_fqdn
project_dict['SPECFEM_BIN'] = cluster_event_bin_fqdn
project_dict['SPECFEM_UTILS'] = cluster_event_utils_fqdn

# Pickle Project Dictionary
f = open(data_root_dir + '/pickled/project_dict.pickle', 'wb')
pickle.dump(project_dict, f)
f.close()

# Test UnPickle Project Dictionary
f = open(data_root_dir + '/pickled/project_dict.pickle', 'rb')
dill_project_dict = pickle.load(f)
f.close()
print()
print('dill_project_dict:\n',dill_project_dict)
print()


# define the access rights
access_rights = 0o755

# create projdir
err = make_directory(proj_fqdn)
if err != 0:
    print('Oops. You messed up creating Project directory!')
    assert False
print()

# create symlinks for projdir
src = cluster_proj_pyutils_fqdn
dst = proj_fqdn + '/pyutils'
if not os.path.islink(dst):
    os.symlink(src, dst)

src = cluster_proj_scriptutils_fqdn
dst = proj_fqdn + '/scriptutils'
if not os.path.islink(dst):
    os.symlink(src, dst)

#loop over number of events and create directories
for e in range(num_events):
    edir = proj_fqdn + '/run' + str(e+1).zfill(4)
    dname = edir
    
    err = make_directory(edir)
    if err != 0:
        print('Oops. You messed up creating run/event dirs!')
        assert False
            
    src = cluster_event_bin_fqdn
    dst = edir + '/bin'
    if not os.path.islink(dst):
        os.symlink(src, dst)

    src = cluster_event_utils_fqdn
    dst = edir + '/utils'
    if not os.path.islink(dst):
        os.symlink(src, dst)
    
    # make for each event
    print('Experiment-%d:\n'%(e+1), dname)
    for d in common_dir_list:
        new_dir = dname + d
        err = make_directory(new_dir)
        if err != 0:
            print('Oops. You messed up creating common dirs!')
            assert False
        
    # make dirs needed only in the primary run0001 dir (used for inversion, model-updating, etc.)
    if e == 0:
        for d in primary_dir_list:
            new_dir = dname + d
            err = make_directory(new_dir)
            if err != 0:
                print('Oops. You messed up creating special dirs!')
                assert False


## Finished